In [ ]:
%matplotlib widget

In [ ]:
from datetime import datetime, timedelta
import calendar
from glob import glob
import json
import pathlib

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

In [ ]:
data = []

for file in sorted(glob('StreamingHistory*.json')):
    with open(file) as fobj:
        data.extend(json.load(fobj))

streaming_history = pd.DataFrame(data)
streaming_history['endTime'] = pd.to_datetime(streaming_history['endTime'])
streaming_history["date"] = streaming_history["endTime"].dt.floor('d')

In [ ]:
# print('Number of entries:', len(streaming_history))

In [ ]:
by_date = streaming_history.groupby("date")[["trackName"]].count()
by_date = by_date.sort_index()

by_date["weekday"] = by_date.index.weekday
by_date["week"] = by_date.index.week

In [ ]:
week = 0
prev_week = by_date.iloc[0]["week"]
continuous_week = np.zeros(len(by_date)).astype(int)
sunday_dates = []
for i, (_, row) in enumerate(by_date.iterrows()):
    if row["week"] != prev_week:
        week += 1
        prev_week = row["week"]
    continuous_week[i] = week
by_date["continuous_week"] = continuous_week

In [ ]:
songs = np.full((7, continuous_week.max()+1), np.nan)

for index, row in by_date.iterrows():
    songs[row["weekday"]][row["continuous_week"]] = row["trackName"]

In [ ]:
min_date = streaming_history["endTime"].min()
first_monday = min_date - timedelta(min_date.weekday())
mons = [first_monday + timedelta(weeks=wk) for wk in range(continuous_week.max())]
x_labels = [calendar.month_abbr[mons[0].month]]
x_labels.extend([
    calendar.month_abbr[mons[i].month] if mons[i-1].month != mons[i].month else "" 
    for i in range(1, len(mons))])

y_labels = ["Mon", "", "Wed", "", "Fri", "", "Sun"]

In [ ]:
x_labels

In [ ]:
songs = np.nan_to_num(songs)

In [ ]:
fig = plt.figure(figsize=(10, 3))
fig.canvas.toolbar_visible = False
ax = plt.subplot()

ax.set_title("My year on Spotify", fontsize=20,pad=40)
ax.xaxis.tick_top()
ax.tick_params(axis='both', which='both',length=0)
ax.set_facecolor("#ebedf0")
fig.patch.set_facecolor('white')

sns.heatmap(songs, linewidths=2, linecolor='white', square=True,
            cmap="Greens", cbar=True,
            vmin=0, vmax=np.max(songs), ax=ax)

ax.set_yticklabels(y_labels, rotation=0)
ax.set_xticklabels(x_labels, ha="left");